# Autoflow

Autoflow is a RAG framework supported:

- Vector Search Based RAG
- Knowledge Graph Based RAG (aka. GraphRAG)
- Knowledge Base and Document Management

## Installation

In [ ]:
%pip install uv

In [4]:
%pip install autoflow-ai==0.0.1.dev33
%pip install autoflow-ai[experiment]==0.0.1.dev33
%pip install ipywidgets

  Attempting uninstall: autoflow-ai
    Found existing installation: autoflow-ai 0.0.1.dev32
    Uninstalling autoflow-ai-0.0.1.dev32:
      Successfully uninstalled autoflow-ai-0.0.1.dev32
Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: autoflow-ai[experiment]==0.0.1.dev33
Note: you may need to restart the kernel to use updated packages.
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)
Note: you may need to restart the kernel to use updated packages.


## Prerequisites

- Go [tidbcloud.com](https://tidbcloud.com/) or using [tiup playground](https://docs.pingcap.com/tidb/stable/tiup-playground/) to create a free TiDB database cluster.
- Go [OpenAI platform](https://platform.openai.com/api-keys) to create your API key.

#### For Jupyter Notebook

Configuration can be provided through environment variables, or using `.env`:

In [ ]:
# Create .env file, then edit your .env, for example:
# $ cat .env
# TIDB_HOST=localhost
# TIDB_PORT=4000
# TIDB_USERNAME=root
# TIDB_PASSWORD=
# OPENAI_API_KEY='your_openai_api_key'
%cp .env.example .env

In [1]:
import os
import dotenv

dotenv.load_dotenv()

True

#### For Google Colab

In [ ]:
from google.colab import userdata

os.environ["TIDB_HOST"] = userdata.get("TIDB_HOST")
os.environ["TIDB_PORT"] = userdata.get("TIDB_PORT")
os.environ["TIDB_USERNAME"] = userdata.get("TIDB_USERNAME")
os.environ["TIDB_PASSWORD"] = userdata.get("TIDB_PASSWORD")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## Quickstart

### Init Autoflow

In [2]:
import os
from autoflow import Autoflow

af = Autoflow.from_config(
    db_host=os.getenv("TIDB_HOST"),
    db_port=int(os.getenv("TIDB_PORT")),
    db_username=os.getenv("TIDB_USERNAME"),
    db_password=os.getenv("TIDB_PASSWORD"),
    db_name=os.getenv("TIDB_DATABASE"),
)

### Create knowledge base

In [3]:
from uuid import UUID
from autoflow.types import IndexMethod
from autoflow.llms.chat_models import ChatModel
from autoflow.llms.embeddings import EmbeddingModel

chat_model = ChatModel("gpt-4o-mini")
embed_model = EmbeddingModel(
    model_name="text-embedding_models-3-small", dimensions=1536
)

kb = af.create_knowledge_base(
    id=UUID("655b6cf3-8b30-4839-ba8b-5ed3c502f30e"),
    name="New KB",
    description="This is a knowledge base for testing",
    index_methods=[IndexMethod.VECTOR_SEARCH, IndexMethod.KNOWLEDGE_GRAPH],
    chat_model=chat_model,
    embedding_model=embed_model,
)
kb.model_dump()

ModuleNotFoundError: No module named 'autoflow.models.entity'

### Import documents from files

In [8]:
current_dir = os.path.dirname(os.path.abspath("__file__"))
current_dir

'/Users/liangzhiyuan/Projects/autoflow.ai/core/docs'

In [9]:
from pathlib import Path

kb.add(
    files=[
        Path(current_dir) / "fixtures" / "tidb-overview.md",
    ]
)

[]

### Search Documents

In [10]:
result = kb.search_documents(
    query="What is TiDB?",
    similarity_top_k=2,
)
[(c.score, c.chunk.text) for c in result.chunks]

[(0.7382171054172685,
  'What is TiDB Self-Managed Key features\n<!-- Localization note for TiDB:\n- English: use distributed SQL, and start to emphasize HTAP\n- Chinese: can keep "NewSQL" and emphasize one-stop real-time HTAP ("一栈式实时 HTAP")\n- Japanese: use NewSQL because it is well-recognized\n-->\nTiDB (/\'taɪdiːbi:/, "Ti" stands for Titanium) is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads. It is MySQL compatible and features horizontal scalability, strong consistency, and high availability. The goal of TiDB is to provide users with a one-stop database solution that covers OLTP (Online Transactional Processing), OLAP (Online Analytical Processing), and HTAP services. TiDB is suitable for various use cases that require high availability and strong consistency with large-scale data.\nTiDB Self-Managed is a product option of TiDB, where users or organizations can deploy and manage TiDB on their own infrastructure

### Search Knowledge Graph

In [11]:
kg = kb.search_knowledge_graph(
    query="What is TiDB?",
)
[(r.rag_description) for r in kg.relationships]

/Users/liangzhiyuan/Projects/autoflow.ai/core/.venv/lib/python3.12/site-packages/sqlmodel/orm/session.py:66: SAWarning: SELECT statement has a cartesian product between FROM element(s) "candidates" and FROM element "entities_655b6cf3-8b30-4839-ba8b-5ed3c502f30e".  Apply join condition(s) between each element to resolve.
  results = super().execute(


['TiDB -> TiDB can be deployed in a Self-Managed model, providing users with control over their database setup. -> Self-Managed',
 'TiDB -> TiDB Self-Managed is a deployment option of TiDB that allows users to manage the database on their own infrastructure. -> TiDB Self-Managed',
 'TiDB -> TiDB utilizes TiKV as its row-based storage engine to support real-time data replication. -> TiKV',
 'TiDB -> TiDB employs TiFlash as its columnar storage engine to ensure consistent data storage and real-time replication from TiKV. -> TiFlash',
 'TiDB -> TiDB Operator facilitates the management of TiDB on Kubernetes, automating operational tasks. -> TiDB Operator',
 'TiDB -> TiDB Cloud is the fully-managed service that allows users to deploy and run TiDB clusters in the cloud. -> TiDB Cloud',
 'TiDB Self-Managed -> TiDB Self-Managed utilizes TiDB Architecture to structure its deployment and operations. -> TiDB Architecture',
 'TiDB Self-Managed -> TiDB Self-Managed relies on TiDB Storage for data m